In [1]:
import functools
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

169017344/169001437 [==============================] - 4s 0us/step


In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 100),
 (10000, 32, 32, 3),
 (10000, 100),
 (10000, 32, 32, 3),
 (10000, 100))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def alexnet(activation):
    AlexNet = Sequential()

    AlexNet.add(Conv2D(filters=96, input_shape=(32, 32, 3), kernel_size=(11, 11), strides=(4, 4), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Flatten())
    AlexNet.add(Dense(4096, input_shape=(32, 32, 3)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(100))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))

    return AlexNet

In [7]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = alexnet('gelu')

batch_size = 128
epochs = 50

top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy', top3_acc, 'top_k_categorical_accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 76s 83ms/step - loss: 4.7744 - accuracy: 0.0216 - top3_acc: 0.0601 - top_k_categorical_accuracy: 0.0945 - val_loss: 4.2972 - val_accuracy: 0.0650 - val_top3_acc: 0.1553 - val_top_k_categorical_accuracy: 0.2244
Epoch 2/50
313/313 [==============================] - 25s 80ms/step - loss: 4.1961 - accuracy: 0.0644 - top3_acc: 0.1519 - top_k_categorical_accuracy: 0.2191 - val_loss: 3.9117 - val_accuracy: 0.1282 - val_top3_acc: 0.2600 - val_top_k_categorical_accuracy: 0.3468
Epoch 3/50
313/313 [==============================] - 25s 81ms/step - loss: 3.9577 - accuracy: 0.0929 - top3_acc: 0.2106 - top_k_categorical_accuracy: 0.2958 - val_loss: 3.7518 - val_accuracy: 0.1469 - val_top3_acc: 0.2901 - val_top_k_categorical_accuracy: 0.3786
Epoch 4/50
313/313 [==============================] - 23s 74ms/step - loss: 3.7944 - accuracy: 0.1230 - top3_acc: 0.2625 - top_k_categorical_accuracy: 0.3539 - val_loss: 3.6576 - val_accuracy: 0.1732 - val_to

In [8]:
print(history.history)
# y_pred = np.argmax(model.predict(x_test), axis=1)
# y_true = np.argmax(y_test,axis=1)

# print(y_pred.shape)
# print(y_true.shape)

# print(np.sum(y_pred == y_true) / y_pred.shape[0])

print(model.evaluate(x_test, y_test))

{'loss': [4.577971935272217, 4.125158786773682, 3.9156336784362793, 3.770704984664917, 3.6495139598846436, 3.5513503551483154, 3.462519645690918, 3.3650577068328857, 3.290083885192871, 3.206914186477661, 3.1254959106445312, 3.051419258117676, 2.9741334915161133, 2.912203788757324, 2.8494646549224854, 2.779911756515503, 2.7125155925750732, 2.65631103515625, 2.5903444290161133, 2.5343430042266846, 2.4705111980438232, 2.42101788520813, 2.3588674068450928, 2.297762393951416, 2.2390525341033936, 2.1805317401885986, 2.1364455223083496, 2.0756049156188965, 2.016026496887207, 1.9709209203720093, 1.9044780731201172, 1.8487820625305176, 1.7968496084213257, 1.7315454483032227, 1.6774678230285645, 1.620581030845642, 1.5654375553131104, 1.5014851093292236, 1.4567039012908936, 1.3950740098953247, 1.3450119495391846, 1.2870584726333618, 1.2385061979293823, 1.184794545173645, 1.1295510530471802, 1.0766576528549194, 1.0379793643951416, 0.9825602769851685, 0.9427459836006165, 0.8941783308982849], 'accur